In [1]:
# main.py
import os
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
if not os.path.exists("plots"):
    os.makedirs("plots")

# ------------------ USER INPUT ------------------ #
tickers = ['AAPL', 'MSFT', 'TSLA', 'AMZN', 'GOOGL']  # You can change these
start_date = "2020-01-01"
end_date = "2024-12-31"

# ------------------ DATA DOWNLOAD ------------------ #
print("Downloading stock data...")
raw_data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker', auto_adjust=False)
data = pd.DataFrame({ticker: raw_data[ticker]['Adj Close'] for ticker in tickers})
data.to_csv("stock_data.csv")
print("Data downloaded.\n")

# ------------------ RETURNS CALCULATION ------------------ #
returns = data.pct_change().dropna()

# ------------------ EDA: CORRELATION ------------------ #
plt.figure(figsize=(10, 6))
sns.heatmap(returns.corr(), annot=True, cmap="coolwarm")
plt.title("Stock Return Correlation")
plt.savefig("plots/correlation_matrix.png")
plt.close()

# ------------------ MONTE CARLO SIMULATION ------------------ #
print("Running portfolio simulation...")

num_portfolios = 10000
all_weights = np.zeros((num_portfolios, len(tickers)))
ret_arr = np.zeros(num_portfolios)
vol_arr = np.zeros(num_portfolios)
sharpe_arr = np.zeros(num_portfolios)

for i in range(num_portfolios):
    weights = np.random.random(len(tickers))
    weights /= np.sum(weights)
    all_weights[i] = weights
    ret_arr[i] = np.sum(returns.mean() * weights * 252)
    vol_arr[i] = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))
    sharpe_arr[i] = ret_arr[i] / vol_arr[i]

plt.figure(figsize=(12, 6))
plt.scatter(vol_arr, ret_arr, c=sharpe_arr, cmap='viridis')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel("Volatility")
plt.ylabel("Return")
plt.title("Monte Carlo Portfolio Simulation")
plt.savefig("plots/monte_carlo.png")
plt.close()

# ------------------ PORTFOLIO OPTIMIZATION ------------------ #
print("Optimizing portfolio (using SciPy)...")

#  EDITED: Helper functions for optimization
def portfolio_performance(weights, mean_returns, cov_matrix):
    ret = np.dot(weights, mean_returns) * 252
    vol = np.sqrt(weights.T @ cov_matrix @ weights) * np.sqrt(252)
    sharpe = ret / vol
    return ret, vol, sharpe

def neg_sharpe(weights, mean_returns, cov_matrix):
    return -portfolio_performance(weights, mean_returns, cov_matrix)[2]

def max_sharpe_portfolio(mean_returns, cov_matrix, num_assets):
    args = (mean_returns, cov_matrix)
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = tuple((0, 1) for _ in range(num_assets))
    result = minimize(neg_sharpe, num_assets * [1/num_assets], args=args,
                      method='SLSQP', bounds=bounds, constraints=constraints)
    return result
mean_returns = returns.mean()
cov_matrix = returns.cov()
result = max_sharpe_portfolio(mean_returns, cov_matrix, len(tickers))
opt_weights = result.x
ret, vol, sharpe = portfolio_performance(opt_weights, mean_returns, cov_matrix)

# Show optimized weights
print("\nOptimized Weights:")
for t, w in zip(tickers, opt_weights):
    print(f"{t}: {w:.2%}")

print(f"\nExpected Annual Return: {ret:.2%}")
print(f"Annual Volatility: {vol:.2%}")
print(f"Sharpe Ratio: {sharpe:.2f}")

# ------------------ VISUALIZE ALLOCATION ------------------ #
plt.figure(figsize=(10, 6))
plt.bar(tickers, opt_weights, color='teal')
plt.title("Optimized Portfolio Allocation")
plt.ylabel("Allocation %")
plt.savefig("plots/optimized_allocation.png")
plt.close()

print("\nAll plots saved in 'plots/' folder.")


[*********************100%***********************]  5 of 5 completed


Data downloaded.

Running portfolio simulation...
Optimizing portfolio (using SciPy)...

Optimized Weights:
AAPL: 40.38%
MSFT: 0.00%
TSLA: 40.75%
AMZN: 0.00%
GOOGL: 18.86%

Expected Annual Return: 48.07%
Annual Volatility: 39.05%
Sharpe Ratio: 1.23

All plots saved in 'plots/' folder.
